In [11]:
import os
import glob
from keras.preprocessing.image import load_img, img_to_array
from keras.utils import to_categorical
import pandas as pd
import numpy as np

folder_path = r"C:\Users\IMS\Desktop\Hwangsihoon\WebCam\Pupil\crop_img"
results = pd.read_excel("results.xlsx", header=None)

files = glob.glob(os.path.join(folder_path, '*.jpg'))

# right파일, left파일 각각 분리
right_images = [file for file in files if 'right' in os.path.basename(file).lower()]
left_images = [file for file in files if 'left' in os.path.basename(file).lower()]


def load_and_preprocess_image(image_path, target_size=(64, 64)):
    image = load_img(image_path, target_size=target_size)
    image_array = img_to_array(image)
    image_array /= 255.0
    return image_array


# First & Sceond Inpput
right_images = [load_and_preprocess_image(img_path) for img_path in right_images]
left_images = [load_and_preprocess_image(img_path) for img_path in left_images]

labels = results

print(f"Number of right images: {len(right_images)}")
print(f"Number of left images: {len(left_images)}")
print(f"Number of labels: {len(labels)}")


Number of right images: 3496
Number of left images: 3496
Number of labels: 3496


In [13]:
import os
import glob
from keras.preprocessing.image import load_img, img_to_array
from keras.utils import to_categorical
import pandas as pd
import numpy as np

folder_path = r"C:\Users\IMS\Desktop\Hwangsihoon\WebCam\Pupil\crop_img"
results = pd.read_excel("results.xlsx", header=None)

files = glob.glob(os.path.join(folder_path, '*.jpg'))

# right파일, left파일 각각 분리
right_images = [file for file in files if 'right' in os.path.basename(file).lower()]
left_images = [file for file in files if 'left' in os.path.basename(file).lower()]


def load_and_preprocess_image(image_path, target_size=(64, 64)):
    image = load_img(image_path, target_size=target_size)
    image_array = img_to_array(image)
    image_array /= 255.0
    return image_array


# First & Sceond Inpput
right_images = [load_and_preprocess_image(img_path) for img_path in right_images]
left_images = [load_and_preprocess_image(img_path) for img_path in left_images]

labels = results

# 원핫인코딩
num_classes = len(np.unique(results))
labels = to_categorical(labels, num_classes=num_classes)

from sklearn.model_selection import train_test_split

right_images_train, right_images_test, left_images_train, left_images_test, labels_train, labels_test = train_test_split(
    right_images, left_images, labels, test_size=0.2, shuffle=True, random_state=42
)

import tensorflow as tf
from keras import layers, Model, Input

def mish(x):
    return x * (tf.math.tanh(tf.math.softplus(x)))

x1 = right_images  # 이미지
x2 = left_images

y1 = labels  # 레이블
y2 = labels

# input1 모델
input1 = Input(shape=(128, 128, 3))
x1 = layers.Conv2D(16, kernel_size=(7, 7), activation=mish, padding='same', strides=(2, 2))(input1)
x1 = layers.BatchNormalization()(x1)
x1 = layers.MaxPooling2D(pool_size=(2, 2))(x1)

x1 = layers.Conv2D(16, kernel_size=(5, 5), activation=mish, padding='same')(x1)
x1 = layers.BatchNormalization()(x1)
x1 = layers.MaxPooling2D(pool_size=(2, 2))(x1)

x1 = layers.Conv2D(32, kernel_size=(3, 3), activation=mish, padding='same')(x1)
x1 = layers.BatchNormalization()(x1)
x1 = layers.MaxPooling2D(pool_size=(2, 2))(x1)

x1 = layers.Conv2D(64, kernel_size=(3, 3), activation=mish, padding='same')(x1)
x1 = layers.BatchNormalization()(x1)
x1 = layers.MaxPooling2D(pool_size=(2, 2))(x1)

x1 = layers.Conv2D(64, kernel_size=(3, 3), activation=mish, padding='same')(x1)
x1 = layers.BatchNormalization()(x1)
x1 = layers.MaxPooling2D(pool_size=(2, 2))(x1)

x1 = layers.Conv2D(128, kernel_size=(2, 2), activation=mish, padding='same')(x1)
x1 = layers.BatchNormalization()(x1)
x1 = layers.MaxPooling2D(pool_size=(2, 2))(x1)

x1 = layers.Flatten()(x1)  # Flatten 레이어

combined = layers.Concatenate()([x1, x2])

# Fully Connected 층
fc_output = layers.Dense(128, activation=mish)(combined)
fc_output = layers.BatchNormalization()(fc_output)
final_output = layers.Dense(5, activation='softmax')(fc_output)

model = Model(inputs=[input1, input2], outputs=final_output)

model.summary()

model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# 모델 학습
history = model.fit([right_images_train, left_images_train], labels_train, 
                    epochs=10, 
                    batch_size=32, 
                    validation_split=0.2)

# 테스트 데이터 평가
test_loss, test_accuracy = model.evaluate([right_images_test, left_images_test], labels_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# 예측
predictions = model.predict([right_images_test, left_images_test])
predicted_classes = np.argmax(predictions, axis=1)

# 예측 결과 확인
for i, pred in enumerate(predicted_classes):
    print(f"Image {i}: Predicted Class: {pred}")

ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concatenation axis. Received: input_shape=[(None, 128), [(64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3), (64, 64, 3),

In [16]:
import os
import glob
from keras.preprocessing.image import load_img, img_to_array
import pandas as pd
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras import layers, Model, Input

# 데이터 로드 및 전처리
folder_path = r"C:\Users\IMS\Desktop\Hwangsihoon\WebCam\Pupil\crop_img"
results = pd.read_excel("results.xlsx", header=None)

files = glob.glob(os.path.join(folder_path, '*.jpg'))

# right파일, left파일 각각 분리
right_images = [file for file in files if 'right' in os.path.basename(file).lower()]
left_images = [file for file in files if 'left' in os.path.basename(file).lower()]

def load_and_preprocess_image(image_path, target_size=(128, 128)):
    image = load_img(image_path, target_size=target_size)
    image_array = img_to_array(image)
    image_array /= 255.0
    return image_array

# 이미지 로딩
right_images = np.array([load_and_preprocess_image(img_path) for img_path in right_images])
left_images = np.array([load_and_preprocess_image(img_path) for img_path in left_images])

# 레이블 준비
labels = results[0].values  # 첫 번째 열을 레이블로 사용한다고 가정
num_classes = len(np.unique(labels))
labels = to_categorical(labels, num_classes=num_classes)

# 데이터 셋 분할
right_images_train, right_images_test, left_images_train, left_images_test, labels_train, labels_test = train_test_split(
    right_images, left_images, labels, test_size=0.2, shuffle=True, random_state=42
)

def create_cnn_model(input_shape):
    input_layer = Input(shape=input_shape)
    x = layers.Conv2D(16, kernel_size=(7, 7), activation='mish', padding='same', strides=(2, 2))(input_layer)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = layers.Conv2D(16, kernel_size=(5, 5), activation='mish', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = layers.Conv2D(32, kernel_size=(3, 3), activation='mish', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = layers.Conv2D(64, kernel_size=(3, 3), activation='mish', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = layers.Conv2D(64, kernel_size=(3, 3), activation='mish', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = layers.Conv2D(128, kernel_size=(2, 2), activation='mish', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    return Model(inputs=input_layer, outputs=x)

# 모델 생성
cnn_right = create_cnn_model((128, 128, 3))
cnn_left = create_cnn_model((128, 128, 3))

# Flatten layers
right_flatten = layers.Flatten()(cnn_right.output)
left_flatten = layers.Flatten()(cnn_left.output)

# Concatenate
combined = layers.Concatenate()([right_flatten, left_flatten])

# Fully Connected 층
fc_output = layers.Dense(128, activation='mish')(combined)
fc_output = layers.BatchNormalization()(fc_output)
final_output = layers.Dense(num_classes, activation='softmax')(fc_output)

# 최종 모델
model = Model(inputs=[cnn_right.input, cnn_left.input], outputs=final_output)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
history = model.fit([right_images_train, left_images_train], labels_train, 
                    epochs=10,  # 원하는 에폭 수로 변경
                    batch_size=32, 
                    validation_split=0.2)  # 학습 데이터의 20%를 검증 데이터로 사용


Epoch 1/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 17s 140ms/step - accuracy: 0.8880 - loss: 0.3208 - val_accuracy: 0.2268 - val_loss: 2.2054
Epoch 2/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 9s 132ms/step - accuracy: 0.9862 - loss: 0.0410 - val_accuracy: 0.2268 - val_loss: 2.8004
Epoch 3/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 9s 128ms/step - accuracy: 0.9869 - loss: 0.0405 - val_accuracy: 0.2268 - val_loss: 2.7397
Epoch 4/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 9s 129ms/step - accuracy: 0.9915 - loss: 0.0287 - val_accuracy: 0.2321 - val_loss: 1.6626
Epoch 5/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 9s 128ms/step - accuracy: 0.9911 - loss: 0.0410 - val_accuracy: 0.4232 - val_loss: 2.4493
Epoch 6/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 9s 128ms/step - accuracy: 0.9993 - loss: 0.0043 - val_accuracy: 0.9643 - val_loss: 0.1330
Epoch 7/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 9s 127ms/step - accuracy: 1.0000 - loss: 0.0013 - val_accuracy: 0.9946 - val_loss: 0.0409
Epoch 8/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 9s 128ms/step - accuracy: 1.0000 - loss: 8.7368e-04 - val_accurac